In [70]:
import pandas as pd
import numpy as np
import findspark
import glob
import os
import re
import nltk
import pickle
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [2]:
import spacy

# Inicialización de SPARK

In [3]:
import findspark
localizacion_spark = '/opt/spark-2.4.5'
findspark.init(localizacion_spark)

import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession

# numero de cores: 4, memoria ram que se le permite a spark usar: 7GB
spark_configurations = SparkConf()\
    .setMaster('local[4]')\
    .setAppName('Tarea_1')\
    .set("spark.driver.memory", "7g")

sc = pyspark.SparkContext(conf = spark_configurations)


spark = SparkSession\
    .builder\
    .master('local[4]')\
    .appName("Tarea_1") \
    .getOrCreate()

# Lectura de datos

In [7]:
def documentReaderSpark(data_path:str, sparkContext):
    """
    DocString
    :return: Nothing
    """
    documents = sc\
        .wholeTextFiles(data_path,
                        minPartitions=None, 
                        use_unicode=True)\
        .map(lambda s: (re.search('<public publicId="(.*?)" uri="(.*?)" />',s[1]).group(1),
                        s[1].replace("\n","")\
                            .replace("\xa0"," "))
            )\
        .map(lambda s: (int(s[0].replace('d','')),re.search('<raw><!\[CDATA\[(.*?)\]\]></raw>',s[1]).group(1)))
    print(documents.collect()[0])
    return documents
documents_path = os.path.join('docs', 'docs-raw-texts')
documents = documentReaderSpark(documents_path, sc)

(223, 'Well, I Didn’t Know it was Hard – Happy Birthday Ivan Sutherland.Ivan Sutherland’s Sketchpad (1963) Happy Birthday 74th Ivan Sutherland! The American computer scientist and Internet pioneer has received the Turing Award from the Association for Computing Machinery in 1988 for his invention of Sketchpad, an early predecessor to the sort of graphical user interface that has become ubiquitous in personal computers today. Sketchpad could accept constraints and specified relationships among segments and arcs, including the diameter of arcs. It could draw both horizontal and vertical lines and combine them into figures and shapes. Figures could be copied, moved, rotated, or resized, retaining their basic properties. Sketchpad also had the first window-drawing program and clipping algorithm, which allowed zooming. When asked, “How could you possibly have done the first interactive graphics program, the first non-procedural programming language, the first object oriented software system

In [9]:
def tokenizationSpark(documents_rdd, use_spacy=False):
    """
    :param documentos:
    :return:
    """
    if use_spacy:
        nlp_spacy_en = spacy.load('en_core_web_sm')
        nltk_lemmaList = documents_rdd\
            .map(lambda s : (s[0], nlp_spacy_en(s[1])))\
            .flatMap(lambda s : [(lemma,s[0]) for lemma in [token.lemma_ for token in s[1]]
                                 if nlp_spacy_en.vocab[lemma].is_stop == False
                                 and nlp_spacy_en.vocab[lemma].is_punct == False])\
            .map(lambda t : ((t[0], t[1]),1))
    else:
        nltk_stop_words_en = set(nltk.corpus.stopwords.words("english"))
        p_stemmer = nltk.stem.porter.PorterStemmer()
        wordnet_lemmatizer = nltk.stem.WordNetLemmatizer()

        nltk_lemmaList = documents_rdd\
            .map(lambda s : (s[0], [word for word in nltk.word_tokenize(s[1]) 
                                    if word.isalnum()]))\
            .flatMap(lambda s : [(token,s[0]) for token in s[1] 
                                 if token not in nltk_stop_words_en])\
            .map(lambda s : ((wordnet_lemmatizer.lemmatize(s[0]), s[1]),1))
    
    print(nltk_lemmaList.filter(lambda x : 223==x[0][1]).collect())
    return nltk_lemmaList
tokenized_docs = tokenizationSpark(documents)


[(('Well', 223), 1), (('I', 223), 1), (('Didn', 223), 1), (('Know', 223), 1), (('Hard', 223), 1), (('Happy', 223), 1), (('Birthday', 223), 1), (('Ivan', 223), 1), (('Sutherland', 223), 1), (('Sketchpad', 223), 1), (('1963', 223), 1), (('Happy', 223), 1), (('Birthday', 223), 1), (('74th', 223), 1), (('Ivan', 223), 1), (('Sutherland', 223), 1), (('The', 223), 1), (('American', 223), 1), (('computer', 223), 1), (('scientist', 223), 1), (('Internet', 223), 1), (('pioneer', 223), 1), (('received', 223), 1), (('Turing', 223), 1), (('Award', 223), 1), (('Association', 223), 1), (('Computing', 223), 1), (('Machinery', 223), 1), (('1988', 223), 1), (('invention', 223), 1), (('Sketchpad', 223), 1), (('early', 223), 1), (('predecessor', 223), 1), (('sort', 223), 1), (('graphical', 223), 1), (('user', 223), 1), (('interface', 223), 1), (('become', 223), 1), (('ubiquitous', 223), 1), (('personal', 223), 1), (('computer', 223), 1), (('today', 223), 1), (('Sketchpad', 223), 1), (('could', 223), 1), (

In [6]:
tokenized_docs.saveAsPickleFile?

In [71]:
def makeInvertedIndexSpark(tokenized_documents_rdd):
    """
    :param documentos:
    :return:
    """
    inverted_index = tokenized_documents_rdd\
        .reduceByKey(lambda a, b : a+b )\
        .map(lambda s : (s[0][0], [[s[0][1],s[1]]]))\
        .reduceByKey(lambda a, b : sorted(a+b) )\
        .sortBy(lambda s : s[0])\
        .map(lambda s :  { s[0]: {'freq' : len(s[1]), 'posting':s[1]}})\
        .collect()
    respuesta = {}
    for item in inverted_index:
        respuesta.update(item)
    with open(os.path.join('docs','inverted_index.pkl'), 'wb') as handle:
        pickle.dump(respuesta, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return respuesta
inverted_index = makeInvertedIndexSpark(tokenized_docs)

In [78]:
# se termina la sesion de spark
sc.stop()